# Emotion Recognition Prototype (Colab Ready)
Upload dataset, select preprocessing, choose model, and train/test.

In [ ]:
!pip install nltk scikit-learn matplotlib seaborn ipywidgets --quiet

In [ ]:
import pandas as pd, nltk, string, seaborn as sns, matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
import ipywidgets as widgets
from IPython.display import display
nltk.download('punkt'); nltk.download('stopwords')

In [ ]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys(): dataset_path = fn
df = pd.read_csv(dataset_path)
df.head()

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
def preprocess_text(text, lowercase=True, remove_punct=True, remove_stop=True):
    if lowercase: text = text.lower()
    if remove_punct: text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = nltk.word_tokenize(text)
    if remove_stop: tokens = [w for w in tokens if w not in stop_words]
    return ' '.join(tokens)

In [ ]:
lowercase_chk = widgets.Checkbox(value=True, description='Lowercase')
punct_chk = widgets.Checkbox(value=True, description='Remove Punctuation')
stop_chk = widgets.Checkbox(value=True, description='Remove Stopwords')
btn_apply = widgets.Button(description='Apply Preprocessing')
out = widgets.Output()
def on_apply_clicked(b):
    with out:
        out.clear_output()
        df['clean_text'] = df['content'].astype(str).apply(lambda x: preprocess_text(x, lowercase_chk.value, punct_chk.value, stop_chk.value))
        print('Preprocessing Applied!')
        display(df.head())
btn_apply.on_click(on_apply_clicked)
display(lowercase_chk, punct_chk, stop_chk, btn_apply, out)

In [ ]:
X = df['content']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)
model_dropdown = widgets.Dropdown(options=['Logistic Regression','Naive Bayes','SVM'], description='Model:')
train_btn = widgets.Button(description='Train Model')
out_train = widgets.Output()
def on_train_clicked(b):
    with out_train:
        out_train.clear_output()
        if model_dropdown.value=='Logistic Regression': model=LogisticRegression(max_iter=1000)
        elif model_dropdown.value=='Naive Bayes': model=MultinomialNB()
        else: model=LinearSVC()
        model.fit(X_train_vec,y_train)
        y_pred=model.predict(X_test_vec)
        print('Accuracy:',accuracy_score(y_test,y_pred))
        print(classification_report(y_test,y_pred))
        cm=confusion_matrix(y_test,y_pred)
        sns.heatmap(cm,annot=True,fmt='d'); plt.show()
train_btn.on_click(on_train_clicked)
display(model_dropdown, train_btn, out_train)